In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import sympy as sym

In [5]:
priorsdf = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
paramsdf = pd.DataFrame(columns = ['name','mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['theKBO','obj1','obj2'])

paramsdf['name'] = ['Haumea','Hi\'aka','Namaka']
paramsdf['mass'] = [2,4,6]
paramsdf['sma'] = [3,4,5]
paramsdf['ecc'] = [0.01,0.1,1]
paramsdf['aop'] = [2,4,6]
paramsdf['inc'] = [3,4,5]
paramsdf['lan'] = [0.01,0.1,1]
paramsdf['mea'] = [2,4,6]
paramsdf['j2r2'] = [3,4,5]
paramsdf['c22r2'] = [2,4,6]
paramsdf['spaop'] = [3,4,5]
paramsdf['spinc'] = [0.01,0.1,1]
paramsdf['splan'] = [2,4,6]

priorsdf['mass'] = [0,1,2,1,2,1,1,1,1]
priorsdf['sma'] = [0,1,2,1,2,1,1,1,1]
priorsdf['ecc'] = [2,1,3,1,2,1,1,1,1]
priorsdf['aop'] = [0,1,2,1,2,1,1,1,1]
priorsdf['inc'] = [0,1,2,1,2,1,1,1,1]
priorsdf['lan'] = [1,1,2,1,2,1,1,1,1]
priorsdf['mea'] = [0,1,2,1,2,1,1,1,1]
priorsdf['j2r2'] = [0,1,2,1,2,1,1,1,1]
priorsdf['c22r2'] = [3,1,2,1,2,1,1,1,1]
priorsdf['spaop'] = [0,1,6,1,2,1,1,1,1]
priorsdf['spinc'] = [0,1,2,1,2,1,1,1,1]
priorsdf['splan'] = [0,1,2,1,2,1,1,1,1]

print(paramsdf,priorsdf)

          name  mass  sma   ecc  aop  inc   lan  mea  j2r2  c22r2  spaop  \
theKBO  Haumea     2    3  0.01    2    3  0.01    2     3      2      3   
obj1    Hi'aka     4    4  0.10    4    4  0.10    4     4      4      4   
obj2    Namaka     6    5  1.00    6    5  1.00    6     5      6      5   

        spinc  splan  
theKBO   0.01      2  
obj1     0.10      4  
obj2     1.00      6                    mass  sma  ecc  aop  inc  lan  mea  j2r2  c22r2  spaop  \
dist. shape         0    0    2    0    0    1    0     0      3      0   
uni-low             1    1    1    1    1    1    1     1      1      1   
uni-up              2    2    3    2    2    2    2     2      2      6   
log-uni-low         1    1    1    1    1    1    1     1      1      1   
log-uni-up          2    2    2    2    2    2    2     2      2      2   
norm-cen            1    1    1    1    1    1    1     1      1      1   
norm-spread         1    1    1    1    1    1    1     1      1      1   
log

In [6]:
def mm_priors(priors, params):
    columnList = list(priors)
    combinedPriorProbs = 0
    x = sym.Symbol('x')
    
    probDist = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['PDF'])
   
    #This loop runs through every column in the priors dataframe, and evaluates the probability density
    #function of the specified type.
    for i in columnList: 
        
        if priors[i][0] == 0:
            #Uniform Distribution Shape
            if params[i][0] > priors[i][2] and par4ams[i][0] < priors[i][1]:
                PriorProb = 1
            else:
                PriorProb = 0
            
        elif priors[i][0] == 1:
            #Log-Uniform Distribution Shape
            if params[i][0] > priors[i][2] and par4ams[i][0] < priors[i][1]:
                PriorProb = 1
            else:
                PriorProb = 0
            
        elif priors[i][0] == 2:
            # Normal Distribution Shape
            PriorProb = 1/(priors[i][5]*np.sqrt(np.pi))*np.exp(-1/2*((params[i][0]-priors[i][6])/priors[i][5])**2)
            
        elif priors[i][0] == 3:
            #Log Normal Distribution Shape
            PriorProb = np.log10(1/(priors[i][7]*np.sqrt(np.pi))*np.exp(-1/2*((params[i][0]-priors[i][8])/priors[i][7])**2))
        else:
            print('Invalid input.') 
            
        #Here, add the Prior Probability Density function for this element to the total
        print(PriorProb)
        combinedPriorProbs = combinedPriorProbs + PriorProb
        probDist[i][0] = PriorProb
        
    print(probDist)
    return combinedPriorProbs

In [8]:
from sympy import *

dist = mm_priors(priorsdf, paramsdf)
print(dist)

np.log10(0)


1.0
1.0
0.3456201487657986
1.0
1.0
0.0
1.0
1.0
-0.4657221772986928
0.2
1.0
1.0
    mass sma      ecc aop inc lan mea j2r2     c22r2 spaop spinc splan
PDF    1   1  0.34562   1   1   0   1    1 -0.465722   0.2     1     1
8.079897971467105


C:\Users\annas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log10
  """


-inf